<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/Deploying_Agents_OpenAI_Agents_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Installation

In [7]:
import sys

# !pip install pydantic-ai openai-agents nest-asyncio
# !{sys.executable} -m pip install -q --upgrade "openai==1.99.9"

import sys, importlib

# 1) Install compatible versions into THIS interpreter
!{sys.executable} -m pip install -q --upgrade "openai==1.99.9" "openai-agents"

# 2) Safety shim: if your OpenAI build doesn't export ResponseTextConfigParam, add a stub
responses = importlib.import_module("openai.types.responses")
if not hasattr(responses, "ResponseTextConfigParam"):
    class ResponseTextConfigParam(dict):
        """Fallback stub for Agents SDK import; real type exists in openai<=1.99.x."""
        pass
    responses.ResponseTextConfigParam = ResponseTextConfigParam  # monkey-patch for import

# 3) Now import Agents SDK
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
print("Agents SDK imported OK")

Agents SDK imported OK


### Patching asyncio for allowing nested use of asyncio.run

In [8]:
import nest_asyncio
nest_asyncio.apply()

### Library Imports and OpenAI_API_KEY Initialization

In [9]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner
from pydantic import BaseModel
from google.colab import userdata
import asyncio
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Setting up Agents for Guardrails and Handoffs

In [10]:
class ResearchOutput(BaseModel):
    is_research: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about research.",
    output_type=ResearchOutput,
)

cs_research_agent = Agent(
    name="CS Researcher",
    handoff_description="Specialist agent for Computer Science Research",
    instructions="You provide help with CS research. Explain your reasoning at each step and include examples",
)

bio_research_agent = Agent(
    name="Biology Researcher",
    handoff_description="Specialist agent for Biological Research",
    instructions="You assist with biological research. Explain important events and context clearly.",
)

### Defining Guardrail Trigger

In [11]:
async def research_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(ResearchOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_research,
    )

### Handoff Setup

In [12]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's research question",
    handoffs=[cs_research_agent, bio_research_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=research_guardrail),
    ],
)

### Explicit Tracer Setup

In [14]:
from agents.tracing.setup import GLOBAL_TRACE_PROVIDER
from agents import set_tracing_export_api_key

#set_tracing_export_api_key('sk-proj-PAI0T_-***********************************************')

GLOBAL_TRACE_PROVIDER._multi_processor.force_flush()

### Executing the Agent

In [15]:
async def main():
    result = await Runner.run(triage_agent, "What is Agentic AI?")
    print(result.final_output)

In [17]:
import os, asyncio
from pydantic import BaseModel, Field
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from google.colab import userdata

# Ensure the key is set for Agents SDK
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# ---- Typed output for the guardrail ----
class ResearchOutput(BaseModel):
    is_research: bool = Field(..., description="True if the user message is a research/knowledge-seeking query.")
    confidence: float = Field(..., ge=0, le=1, description="Model's confidence in the classification.")
    reasoning: str = Field(..., description="Short rationale for the classification.")

# ---- Guardrail classifier agent ----
guardrail_agent = Agent(
    name="Guardrail check",
    instructions=(
        "You are a binary classifier for whether the user's message is a RESEARCH query.\n"
        "Definition of research: academic/technical knowledge-seeking (explanations, concepts, literature, "
        "comparisons of methods, hypotheses, taxonomies, background context, summaries of fields).\n\n"
        "Examples that ARE research:\n"
        " - 'What is Agentic AI?'\n"
        " - 'Summarize recent papers on diffusion models'\n"
        " - 'Explain the difference between CNNs and Transformers'\n"
        " - 'What are best practices for RAG evaluation?'\n\n"
        "Examples that are NOT research:\n"
        " - 'Tell me a joke'\n"
        " - 'Translate this sentence' (no research context)\n"
        " - 'Schedule a meeting for me'\n"
        " - 'Write code to scrape this site' (pure tasking, not knowledge seeking)\n\n"
        "Ambiguity policy: If there is any doubt, set is_research=true. "
        "Return ONLY the JSON matching the provided schema."
    ),
    output_type=ResearchOutput,
)

# ---- Specialists ----
cs_research_agent = Agent(
    name="CS Researcher",
    handoff_description="Specialist agent for Computer Science Research",
    instructions="You provide help with CS research. Be clear and structured; include examples when useful."
)

bio_research_agent = Agent(
    name="Biology Researcher",
    handoff_description="Specialist agent for Biological Research",
    instructions="You assist with biological research. Explain important mechanisms and context clearly."
)

# ---- Guardrail function ----
async def research_guardrail(ctx, agent, input_data):
    try:
        result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
        final_output = result.final_output_as(ResearchOutput)
        # Only trip if confidently NON-research
        should_block = (final_output.is_research is False) and (final_output.confidence >= 0.85)
        return GuardrailFunctionOutput(
            output_info=final_output,
            tripwire_triggered=should_block,
        )
    except Exception as e:
        # Fail open: if the classifier has trouble, don't block the user
        fallback = ResearchOutput(is_research=True, confidence=0.0, reasoning=f"Guardrail error: {e}")
        return GuardrailFunctionOutput(output_info=fallback, tripwire_triggered=False)

# ---- Triage agent with guardrail ----
triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You determine which agent to use based on the user's research question. "
        "If the topic is computer science/AI/ML/software, choose CS Researcher. "
        "If biological/life sciences, choose Biology Researcher. "
        "Otherwise answer directly."
    ),
    handoffs=[cs_research_agent, bio_research_agent],
    input_guardrails=[InputGuardrail(guardrail_function=research_guardrail)],
)

# ---- Optional tracing (safe if tracing not configured) ----
try:
    from agents.tracing.setup import GLOBAL_TRACE_PROVIDER
    GLOBAL_TRACE_PROVIDER._multi_processor.force_flush()
except Exception:
    pass

# ---- Demo ----
async def main():
    query = "What is Agentic AI?"
    result = await Runner.run(triage_agent, query)
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())


**Agentic AI** refers to artificial intelligence systems designed to act as **autonomous agents**—entities that can perceive their environment, make decisions, and take actions independently to achieve specific goals. The term “agentic” comes from “agency,” meaning the ability to act and make choices. Let’s break this concept down:

### Key Features of Agentic AI

1. **Autonomy:**  
   Agentic AI operates with a significant degree of independence, making decisions without direct human intervention.

2. **Goal-Directed Behavior:**  
   Agentic AIs are typically designed to pursue objectives, which can range from simple tasks (like organizing emails) to complex ones (like managing supply chains).

3. **Environment Interaction:**  
   Such AIs perceive their world (via sensors, data feeds, etc.) and act on it (sending commands, communicating, or physically manipulating objects).

4. **Learning and Adaptation:**  
   Often, agentic AIs can learn from their environment, adapt strategies, an